# NeuroTracker CDA Analysis - Full Report with Re-Ref to Common Avg

In [1]:
import numpy as np
import pandas as pd
import glob
import pickle

import mne
from mne import Epochs, pick_channels, find_events
from mne.io import read_raw_bdf

import collections
from autoreject import AutoReject

import sys
sys.path.append('/Users/nick/Documents/PhD/CLCT/InLook/NeuroTracker/')
from cda_utils import get_epochs, get_CDA, prep_report, fill_report, get_report, checksum, add_cda_report, get_CDA_perf_report
#from events import get_specific_events

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# 0 - General Info

In [2]:
#==================================================================
# General Variables
#==================================================================
# Organize Triggers
trigger = 2

# Triggers
# 1 - New Spheres.
# 2 - Red Spheres.
# 4 - Start Moving.
# 8 - Stop Moving.
# 16 - TBD (~4 of them... and 4 blocks.)
# 32 - TBD (juste juste avant le 1, so I guess fin de trial, I have 88 of them... 4 trials)
# 

# T2 = T1 + 5s 
# T4 = T2 + 2s
# T8 = T4 + 9s ?! (9s ?)
# T16 = T8 + ~1s
# T32 = T8 + 25s   ensuite un autre 3s plus tard.

# Beaucoup de 1 et 32.. pas clair...

# --------------------------
conds = ['1', '2', '3']
sides = ['right', 'left', 'classicmod']
perfs = ['good', 'bad']

custom_trigger = dict()
custom_trigger['1'] = 100
custom_trigger['2'] = 200
custom_trigger['3'] = 300
custom_trigger['right'] = 10
custom_trigger['left'] = 20
custom_trigger['classicmod'] = 30
custom_trigger['good'] = 1
custom_trigger['bad'] = 2

# Making Event ID Dict to make it easy and readable to access (the desired) epochs.
event_id = dict()
for cond in conds:
    for side in sides:
        for perf in perfs:
            event_id['{}/{}/{}'.format(cond, side, perf)] = custom_trigger[cond] + custom_trigger[side] + custom_trigger[perf]

#report = prep_report(conds, sides, perfs)

# CDA Channels
chan_right_all = ['O2', 'PO4', 'PO8', 'P2', 'P4', 'P6', 'P8', 'P10', 'C2', 'C4', 'C6', 'CP2', 'CP4', 'CP6', 'T8', 'TP8', 'FT8', 'FC2', 'FC4', 'FC6', 'F2', 'F4', 'F6', 'F8', 'AF4', 'AF8', 'Fp2']
chan_left_all =  ['O1', 'PO3', 'PO7', 'P1', 'P3', 'P5', 'P7', 'P9',  'C1', 'C3', 'C5', 'CP1', 'CP3', 'CP5', 'T7', 'TP7', 'FT7', 'FC1', 'FC3', 'FC5', 'F1', 'F3', 'F5', 'F7', 'AF3', 'AF7', 'Fp1']
chan_midline_all = ['Fz', 'FCz', 'Cz', 'Fpz', 'CPz', 'Pz', 'POz', 'Oz'] #'Iz'

# Epoch Information
epoch_length = 20
epoch_tmin = -3
baseline_corr = (-1, -0.6)

# Epochs variable!
epochs_dict_all = dict()
rt_dict_all = dict()

In [5]:
#==================================================================
# Main Loop - Loading All Files & Extract CDAs.
#==================================================================
directory_path = '/Users/nick/Documents/PhD/CAE - InLook/NeuroTracker - Sub/'

#filepath = directory_path + 'M22F2.bdf'
for filepath in glob.iglob(directory_path + '*.bdf'):
    filename = filepath[filepath.rfind('/')+1:]
    print('Processing: ' + filename + '...')

    # 0. -- Prep MNE-Report --
    report = mne.Report(title='Subject Report - {}'.format(filename))
    report_filename = 'Subject_Report-{}.html'.format(filename[:-4])

    # 1. -- Load File --
    raw = read_raw_bdf(filepath, preload=True)

    # 2. -- General Preprocess --
    # Keep only EEG channels and reference
    raw = raw.drop_channels(ch_names=['EXG1','EXG2','EXG3','EXG4','EXG5','EXG6','EXG7', 'EXG8'])
    raw = raw.set_eeg_reference(ref_channels='average')

    # Set Electrodes Locations
    montage = mne.channels.make_standard_montage('biosemi64')
    raw.set_montage(montage)

    # Resample
    preproc = raw.resample(500)

    # Filter
    preproc = preproc.filter(1, 30, fir_design='firwin', skip_by_annotation='edge')

    # ------------------------
    # 3. -- Epochs --
    # ------------------------
    events = find_events(preproc)

    # ---------- Debug Logs ----------
    frequency = collections.Counter(events[:,2])
    events_frequency = dict(frequency)
    print('Max Events in File...')
    print("---------------------------")
    for e in set(events[:,2]):
        print("{}: \t\t{}".format(e, frequency[e]))
    print("---------------------------")
    # ---------- Debug Logs ----------

    df_behav = pd.read_csv(directory_path + filename[:filename.rfind('.')] + '.csv', sep=',')

    new_events = []
    nb_trials = 0
    total_skipped = 0
    for e in events:
        cur_cond = None
        cur_side = None
        cur_perf = None
        if e[2] == 2: # Find Start of Trial
            cur_cond = str(df_behav['NbTargets'].values[nb_trials]).lower()
            cur_side = str(df_behav['Mode'].values[nb_trials]).lower()
            cur_perf = 'good' if (int(df_behav['NbTargets'].values[nb_trials]) == int(df_behav['Results'].values[nb_trials])) else 'bad'
            if cur_cond is not None and cur_side is not None and cur_perf is not None:
                #temp_trigger = int(custom_trigger[cur_cond]) + int(custom_trigger[cur_side]) + int (custom_trigger[cur_perf])
                temp_event = [e[0], e[1], event_id['{}/{}/{}'.format(cur_cond, cur_side, cur_perf)]]
                new_events.append(temp_event) # Make a new list of events with new custom triggers.
            else:
                total_skipped = total_skipped + 1
                print('Skipping this event {}: No Cond or Side or Perf'.format(e))

            nb_trials = nb_trials + 1

            # Check Synch!
            if (nb_trials + 1) % 10 == 0:
                print('Check Synch!')

    print("A total of {} trials were added and {} were skipped.".format(nb_trials, total_skipped))

    #epochs = Epochs(preproc, new_events, event_id=event_id, tmin=epoch_tmin, tmax=epoch_length, baseline=baseline_corr, preload=True, event_repeated='merge', on_missing='warn')
    epochs = Epochs(preproc, new_events, event_id=event_id, tmin=epoch_tmin, tmax=epoch_length, baseline=(None, None), preload=True, event_repeated='merge', on_missing='warn')
    #epochs = get_epochs(preproc, specific_events, epoch_length, epoch_tmin, baseline_corr)

    # Can't do this because the 'on_missing' apparently doesn't follow in bipolar_ref. And because I have missing '1/left/bad'
    #epochs = mne.set_bipolar_reference(epochs, anode=chan_left_all, cathode=chan_right_all, verbose=False)

    # Set Bipolar Channels
    epochs_dict_all[filename] = epochs

    # Calculate average response time.
    resp_time_in_ticks = []
    for i, e in enumerate(events):
        if e[2] == 8:
            if events[i+1,2] == 32:
                resp_time_in_ticks.append(events[i+1,0] - events[i,0])
            elif events[i+1,2] == 16: #Synch?!
                if events[i+2,2] == 32:
                    resp_time_in_ticks.append(events[i+2,0] - events[i,0])
                else:
                    print('No Response?! Trigger: {}'.format(events[i+1,2]))
            else:
                print('No Response?! Trigger: {}'.format(events[i+1,2]))

    mean_rt = np.mean(resp_time_in_ticks)
    std_rt = np.std(resp_time_in_ticks)
    print('Response Time ~{}s (std:{})'.format(mean_rt/500, std_rt/500))

    rt_dict_all[filename] = resp_time_in_ticks

    report.add_epochs(epochs=epochs, title='Epochs')
    #report.save(report_filename, overwrite=True)

    report_sides(report=report, epochs=epochs, chan_right_all=chan_right_all, chan_left_all=chan_left_all, perf='good')
    report_sizes(report=report, epochs=epochs, chan_right_all=chan_right_all, chan_left_all=chan_left_all, perf='all')
    report.save(report_filename, overwrite=True)

Processing: T16F2.bdf...
Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Extracting EDF parameters from /Users/nick/Documents/PhD/CAE - InLook/NeuroTracker - Sub/T16F2.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5064703  =      0.000 ...  2473.000 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
459 events found
Event IDs: [ 1  2  4  8 16 32]
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
459 events found
Event IDs: [ 1  2  4  8 16 3

/var/folders/gn/5c249d_15xb7y6sdl7n8cn9m0000gn/T/ipykernel_83593/440326644.py:78: RuntimeWarning: No matching events found for 1/left/bad (event id 122)
  epochs = Epochs(preproc, new_events, event_id=event_id, tmin=epoch_tmin, tmax=epoch_length, baseline=(None, None), preload=True, event_repeated='merge', on_missing='warn')
/var/folders/gn/5c249d_15xb7y6sdl7n8cn9m0000gn/T/ipykernel_83593/440326644.py:78: RuntimeWarning: No matching events found for 1/classicmod/bad (event id 132)
  epochs = Epochs(preproc, new_events, event_id=event_id, tmin=epoch_tmin, tmax=epoch_length, baseline=(None, None), preload=True, event_repeated='merge', on_missing='warn')
/var/folders/gn/5c249d_15xb7y6sdl7n8cn9m0000gn/T/ipykernel_83593/440326644.py:78: RuntimeWarning: No matching events found for 2/classicmod/bad (event id 232)
  epochs = Epochs(preproc, new_events, event_id=event_id, tmin=epoch_tmin, tmax=epoch_length, baseline=(None, None), preload=True, event_repeated='merge', on_missing='warn')


0 bad epochs dropped
Response Time ~3.4956533333333333s (std:1.6088569316408743)
    Using multitaper spectrum estimation with 7 DPSS windows
Saving report to : /Users/nick/Documents/PhD/CLCT/InLook/NeuroTracker/CDA Full Report/Subject_Report-T16F2.html
Processing: M22F2.bdf...
Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Extracting EDF parameters from /Users/nick/Documents/PhD/CAE - InLook/NeuroTracker - Sub/M22F2.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6383615  =      0.000 ...  3117.000 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
516 events found
Event IDs: [ 1  2  4  8 16 32]
516 events found
Event IDs: [ 1  2  4  8 16 32]
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from

/var/folders/gn/5c249d_15xb7y6sdl7n8cn9m0000gn/T/ipykernel_83593/440326644.py:78: RuntimeWarning: No matching events found for 1/left/bad (event id 122)
  epochs = Epochs(preproc, new_events, event_id=event_id, tmin=epoch_tmin, tmax=epoch_length, baseline=(None, None), preload=True, event_repeated='merge', on_missing='warn')


0 bad epochs dropped
Response Time ~5.159285714285715s (std:4.389798472897482)
    Using multitaper spectrum estimation with 7 DPSS windows
Saving report to : /Users/nick/Documents/PhD/CLCT/InLook/NeuroTracker/CDA Full Report/Subject_Report-M22F2.html
Processing: W17M2.bdf...
Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Extracting EDF parameters from /Users/nick/Documents/PhD/CAE - InLook/NeuroTracker - Sub/W17M2.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7319551  =      0.000 ...  3574.000 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
513 events found
Event IDs: [    1     2     4     8    16    

...

0 bad epochs dropped
Response Time ~7.372595238095238s (std:4.703730482841828)
    Using multitaper spectrum estimation with 7 DPSS windows
Saving report to : /Users/nick/Documents/PhD/CLCT/InLook/NeuroTracker/CDA Full Report/Subject_Report-T23F1.html
Processing: F12M1.bdf...
Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Extracting EDF parameters from /Users/nick/Documents/PhD/CAE - InLook/NeuroTracker - Sub/F12M1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 5881855  =      0.000 ...  2872.000 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 eve

/var/folders/gn/5c249d_15xb7y6sdl7n8cn9m0000gn/T/ipykernel_83593/440326644.py:78: RuntimeWarning: No matching events found for 1/classicmod/bad (event id 132)
  epochs = Epochs(preproc, new_events, event_id=event_id, tmin=epoch_tmin, tmax=epoch_length, baseline=(None, None), preload=True, event_repeated='merge', on_missing='warn')


0 bad epochs dropped
Response Time ~3.2318571428571428s (std:1.6834125427582054)
    Using multitaper spectrum estimation with 7 DPSS windows
Saving report to : /Users/nick/Documents/PhD/CLCT/InLook/NeuroTracker/CDA Full Report/Subject_Report-F12M1.html
Processing: T23F2.bdf...
Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Extracting EDF parameters from /Users/nick/Documents/PhD/CAE - InLook/NeuroTracker - Sub/T23F2.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 6369279  =      0.000 ...  3110.000 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Trigger channel has a non-zero initial value of 65536 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
516 e

/var/folders/gn/5c249d_15xb7y6sdl7n8cn9m0000gn/T/ipykernel_83593/440326644.py:78: RuntimeWarning: No matching events found for 1/classicmod/bad (event id 132)
  epochs = Epochs(preproc, new_events, event_id=event_id, tmin=epoch_tmin, tmax=epoch_length, baseline=(None, None), preload=True, event_repeated='merge', on_missing='warn')


0 bad epochs dropped
Response Time ~6.2246428571428565s (std:6.283510113673682)
    Using multitaper spectrum estimation with 7 DPSS windows
Saving report to : /Users/nick/Documents/PhD/CLCT/InLook/NeuroTracker/CDA Full Report/Subject_Report-T23F2.html


# Finding Best CDA Channel Pair

## Left vs Right

In [4]:
def report_sides(report, epochs, chan_right_all, chan_left_all, perf='good'):
    # ----------------------------
    # Plot Full Epoch/Trial
    # ----------------------------
    fig, axs = plt.subplots(len(chan_right_all), 1, figsize=(15, 3*len(chan_right_all)))

    for i, ch in enumerate(chan_right_all):
        ch_pair = '{}-{}'.format(chan_left_all[i], chan_right_all[i])

        side = 'right'
        e_filter = side + '/' + perf if (perf == 'good' or perf == 'bad') else side
        cur_evoked_right = epochs[e_filter].copy().pick_channels([chan_right_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_evoked_left = epochs[e_filter].copy().pick_channels([chan_left_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_cda_right = (cur_evoked_left.get_data() - cur_evoked_right.get_data())[0]

        side = 'left'
        e_filter = side + '/' + perf if (perf == 'good' or perf == 'bad') else side
        cur_evoked_right = epochs[e_filter].copy().pick_channels([chan_right_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_evoked_left = epochs[e_filter].copy().pick_channels([chan_left_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_cda_left = (cur_evoked_right.get_data() - cur_evoked_left.get_data())[0]

        side = 'classicmod'
        e_filter = side + '/' + perf if (perf == 'good' or perf == 'bad') else side
        cur_evoked_right = epochs[e_filter].copy().pick_channels([chan_right_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_evoked_left = epochs[e_filter].copy().pick_channels([chan_left_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_cda_classic = (cur_evoked_right.get_data() - cur_evoked_left.get_data())[0]
        
        times = cur_evoked_right.times

        axs[i].plot(times, cur_cda_right, label='{} Right'.format(ch_pair), alpha=0.3)
        axs[i].plot(times, cur_cda_left, label='{} Left'.format(ch_pair), alpha=0.3)
        axs[i].plot(times, cur_cda_classic, label='{} Classic'.format(ch_pair), alpha=0.3)
        axs[i].plot(times, (cur_cda_left + cur_cda_right)/2, label='{} Both'.format(ch_pair))
        axs[i].axvline(x=11, color ='k', linestyle='--')
        axs[i].axhline(y=0, color ='k', linestyle='-')
        axs[i].invert_yaxis()
        axs[i].legend()

    fig.tight_layout()
    #plt.show()

    report.add_figure(
    fig=fig, title='L vs R ({}) - Trial'.format(perf.capitalize()),
    caption='L vs R - Full Trial ({} Trials)'.format(perf.capitalize()),
    tags=('Electrodes','Sides',),
    image_format='PNG')


    # ----------------------------
    # Plot 3 Phases Separately
    # ----------------------------
    fig, axs = plt.subplots(len(chan_right_all), 3, figsize=(15, 3*len(chan_right_all)))

    for i, ch in enumerate(chan_right_all):
        ch_pair = '{}-{}'.format(chan_left_all[i], chan_right_all[i])

        side = 'right'
        e_filter = side + '/' + perf if (perf == 'good' or perf == 'bad') else side
        cur_evoked_right = epochs[e_filter].copy().pick_channels([chan_right_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_evoked_left = epochs[e_filter].copy().pick_channels([chan_left_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_cda_right = (cur_evoked_left.get_data() - cur_evoked_right.get_data())[0]

        side = 'left'
        e_filter = side + '/' + perf if (perf == 'good' or perf == 'bad') else side
        cur_evoked_right = epochs[e_filter].copy().pick_channels([chan_right_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_evoked_left = epochs[e_filter].copy().pick_channels([chan_left_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_cda_left = (cur_evoked_right.get_data() - cur_evoked_left.get_data())[0]

        side = 'classicmod'
        e_filter = side + '/' + perf if (perf == 'good' or perf == 'bad') else side
        cur_evoked_right = epochs[e_filter].copy().pick_channels([chan_right_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_evoked_left = epochs[e_filter].copy().pick_channels([chan_left_all[i]]).crop(tmin=-0.4, tmax=20).average()
        cur_cda_classic = (cur_evoked_right.get_data() - cur_evoked_left.get_data())[0]

        times = cur_evoked_right.times    

        time_filter = (times < 2)
        axs[i, 0].plot(times[time_filter], cur_cda_right[time_filter], label='{} Left'.format(ch_pair), alpha=0.3)
        axs[i, 0].plot(times[time_filter], cur_cda_left[time_filter], label='{} Right'.format(ch_pair), alpha=0.3)
        axs[i, 0].plot(times[time_filter], cur_cda_classic[time_filter], label='{} Classic'.format(ch_pair), alpha=0.3)
        axs[i, 0].plot(times[time_filter], (cur_cda_left[time_filter] + cur_cda_right[time_filter])/2, label='{} Both'.format(ch_pair))
        axs[i, 0].axvline(x=0, color='k', linestyle='--')
        axs[i, 0].axhline(y=0, color='k', linestyle='-')
        axs[i, 0].invert_yaxis()
        axs[i, 0].legend()

        time_filter = (times > 1.5) & (times < 10)
        axs[i, 1].plot(times[time_filter], cur_cda_right[time_filter], label='{} Left'.format(ch_pair), alpha=0.3)
        axs[i, 1].plot(times[time_filter], cur_cda_left[time_filter], label='{} Right'.format(ch_pair), alpha=0.3)
        axs[i, 1].plot(times[time_filter], cur_cda_classic[time_filter], label='{} Classic'.format(ch_pair), alpha=0.3)
        axs[i, 1].plot(times[time_filter], (cur_cda_left[time_filter] + cur_cda_right[time_filter])/2, label='{} Both'.format(ch_pair))
        axs[i, 1].axvline(x=2, color='k', linestyle='--')
        axs[i, 1].axhline(y=0, color='k', linestyle='-')
        axs[i, 1].invert_yaxis()
        axs[i, 1].legend()

        time_filter = (times > 10.5) & (times < 18)
        axs[i, 2].plot(times[time_filter], cur_cda_right[time_filter], label='{} Left'.format(ch_pair), alpha=0.3)
        axs[i, 2].plot(times[time_filter], cur_cda_left[time_filter], label='{} Right'.format(ch_pair), alpha=0.3)
        axs[i, 2].plot(times[time_filter], cur_cda_classic[time_filter], label='{} Classic'.format(ch_pair), alpha=0.3)
        axs[i, 2].plot(times[time_filter], (cur_cda_left[time_filter] + cur_cda_right[time_filter])/2, label='{} Both'.format(ch_pair))
        axs[i, 2].axvline(x=11, color='k', linestyle='--')
        axs[i, 2].axhline(y=0, color='k', linestyle='-')
        axs[i, 2].invert_yaxis()
        axs[i, 2].legend()

    fig.tight_layout()
    #plt.show()

    report.add_figure(
    fig=fig, title='L vs R ({}) - Phases'.format(perf.capitalize()),
    caption='L vs R - 3 Phases ({} Trials)'.format(perf.capitalize()),
    tags=('Electrodes','Sides',),
    image_format='PNG')



# Set Size: 1 vs 2 vs 3

In [3]:
def report_sizes(report, epochs, chan_right_all, chan_left_all, perf='good'):

    cur_cda_right = dict()
    cur_cda_left = dict()

    # ----------------------------
    # Plot Full Epoch/Trial
    # ----------------------------
    fig, axs = plt.subplots(len(chan_right_all), 1, figsize=(15, 3*len(chan_right_all)))

    for i, ch in enumerate(chan_right_all):
        ch_pair = '{}-{}'.format(chan_left_all[i], chan_right_all[i])

        for ss in ['1','2','3']:
            side = f'{ss}/right'
            e_filter = side + '/' + perf if (perf == 'good' or perf == 'bad') else side
            cur_evoked_right = epochs[e_filter].copy().pick_channels([chan_right_all[i]]).crop(tmin=-0.4, tmax=20).average()
            cur_evoked_left = epochs[e_filter].copy().pick_channels([chan_left_all[i]]).crop(tmin=-0.4, tmax=20).average()
            #cur_evoked_bipolar = mne.set_bipolar_reference(cur_evoked, anode=[chan_left_all[i]], cathode=[chan_right_all[i]], verbose=False)
            cur_cda_right[ss] = (cur_evoked_left.get_data() - cur_evoked_right.get_data())[0]

            side = f'{ss}/left'
            e_filter = side + '/' + perf if (perf == 'good' or perf == 'bad') else side
            cur_evoked_right = epochs[e_filter].copy().pick_channels([chan_right_all[i]]).crop(tmin=-0.4, tmax=20).average()
            cur_evoked_left = epochs[e_filter].copy().pick_channels([chan_left_all[i]]).crop(tmin=-0.4, tmax=20).average()
            #cur_evoked_bipolar = mne.set_bipolar_reference(cur_evoked, anode=[chan_left_all[i]], cathode=[chan_right_all[i]], verbose=False)
            cur_cda_left[ss] =  (cur_evoked_right.get_data() - cur_evoked_left.get_data())[0]

        times = cur_evoked_right.times

        # Plot whole trial
        axs[i].plot(times, (cur_cda_left['1'] + cur_cda_right['1'])/2, label='1: {} Both'.format(ch_pair))
        axs[i].plot(times, (cur_cda_left['2'] + cur_cda_right['2'])/2, label='2: {} Both'.format(ch_pair))
        axs[i].plot(times, (cur_cda_left['3'] + cur_cda_right['3'])/2, label='3: {} Both'.format(ch_pair))
        axs[i].axvline(x=11, color ='k', linestyle='--')
        axs[i].axhline(y=0, color ='k', linestyle='-')
        axs[i].invert_yaxis()
        axs[i].legend()

    fig.tight_layout()
    #plt.show()

    report.add_figure(
    fig=fig, title='1 vs 2 vs 3 ({}) - Trial'.format(perf.capitalize()),
    caption='1 vs 2 vs 3 ({}) - Full Trial'.format(perf.capitalize()),
    tags=('Electrodes','SetSizes',),
    image_format='PNG')

    # ----------------------------
    # Plot 3 Phases Separately
    # ----------------------------
    fig, axs = plt.subplots(len(chan_right_all), 3, figsize=(15, 3*len(chan_right_all)))

    for i, ch in enumerate(chan_right_all):
        ch_pair = '{}-{}'.format(chan_left_all[i], chan_right_all[i])

        for ss in ['1','2','3']:
            side = f'{ss}/right'
            e_filter = side + '/' + perf if (perf == 'good' or perf == 'bad') else side
            cur_evoked_right = epochs[e_filter].copy().pick_channels([chan_right_all[i]]).crop(tmin=-0.4, tmax=20).average()
            cur_evoked_left = epochs[e_filter].copy().pick_channels([chan_left_all[i]]).crop(tmin=-0.4, tmax=20).average()
            #cur_evoked_bipolar = mne.set_bipolar_reference(cur_evoked, anode=[chan_left_all[i]], cathode=[chan_right_all[i]], verbose=False)
            cur_cda_right[ss] = (cur_evoked_left.get_data() - cur_evoked_right.get_data())[0]

            side = f'{ss}/left'
            e_filter = side + '/' + perf if (perf == 'good' or perf == 'bad') else side
            cur_evoked_right = epochs[e_filter].copy().pick_channels([chan_right_all[i]]).crop(tmin=-0.4, tmax=20).average()
            cur_evoked_left = epochs[e_filter].copy().pick_channels([chan_left_all[i]]).crop(tmin=-0.4, tmax=20).average()
            #cur_evoked_bipolar = mne.set_bipolar_reference(cur_evoked, anode=[chan_left_all[i]], cathode=[chan_right_all[i]], verbose=False)
            cur_cda_left[ss] =  (cur_evoked_right.get_data() - cur_evoked_left.get_data())[0]
        
        times = cur_evoked_right.times

        time_filter = (times < 2)
        axs[i, 0].plot(times[time_filter], (cur_cda_left['1'][time_filter] + cur_cda_right['1'][time_filter])/2, label='1: {} Both'.format(ch_pair))
        axs[i, 0].plot(times[time_filter], (cur_cda_left['2'][time_filter] + cur_cda_right['2'][time_filter])/2, label='2: {} Both'.format(ch_pair))
        axs[i, 0].plot(times[time_filter], (cur_cda_left['3'][time_filter] + cur_cda_right['3'][time_filter])/2, label='3: {} Both'.format(ch_pair))
        axs[i, 0].axvline(x=0, color='k', linestyle='--')
        axs[i, 0].axhline(y=0, color='k', linestyle='-')
        axs[i, 0].invert_yaxis()
        axs[i, 0].legend()

        time_filter = (times > 1.5) & (times < 10)
        axs[i, 1].plot(times[time_filter], (cur_cda_left['1'][time_filter] + cur_cda_right['1'][time_filter])/2, label='1: {} Both'.format(ch_pair))
        axs[i, 1].plot(times[time_filter], (cur_cda_left['2'][time_filter] + cur_cda_right['2'][time_filter])/2, label='2: {} Both'.format(ch_pair))
        axs[i, 1].plot(times[time_filter], (cur_cda_left['3'][time_filter] + cur_cda_right['3'][time_filter])/2, label='3: {} Both'.format(ch_pair))
        axs[i, 1].axvline(x=2, color='k', linestyle='--')
        axs[i, 1].axhline(y=0, color='k', linestyle='-')
        axs[i, 1].invert_yaxis()
        axs[i, 1].legend()

        time_filter = (times > 10.5) & (times < 18)
        axs[i, 2].plot(times[time_filter], (cur_cda_left['1'][time_filter] + cur_cda_right['1'][time_filter])/2, label='1: {} Both'.format(ch_pair))
        axs[i, 2].plot(times[time_filter], (cur_cda_left['2'][time_filter] + cur_cda_right['2'][time_filter])/2, label='2: {} Both'.format(ch_pair))
        axs[i, 2].plot(times[time_filter], (cur_cda_left['3'][time_filter] + cur_cda_right['3'][time_filter])/2, label='3: {} Both'.format(ch_pair))
        axs[i, 2].axvline(x=11, color='k', linestyle='--')
        axs[i, 2].axhline(y=0, color='k', linestyle='-')
        axs[i, 2].invert_yaxis()
        axs[i, 2].legend()

    fig.tight_layout()
    #plt.show()

    report.add_figure(
    fig=fig, title='1 vs 2 vs 3 ({}) - Phases'.format(perf.capitalize()),
    caption='1 vs 2 vs 3 ({}) - 3 Phases'.format(perf.capitalize()),
    tags=('Electrodes','SetSizes',),
    image_format='PNG')

In [5]:
report.save(report_filename, overwrite=True)

NameError: name 'report' is not defined